In [1]:
import os
import sys
import re
import shutil
import time
import pickle
from collections import defaultdict
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from pandas.plotting import register_matplotlib_converters
#register_matplotlib_converters()


Bad key text.latex.unicode in file c:\users\abhishek buragohaibn\anaconda3\envs\computervison\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 112 ('text.latex.unicode : False # use "ucs" and "inputenc" LaTeX packages for handling')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.4/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.frameon in file c:\users\abhishek buragohaibn\anaconda3\envs\computervison\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 423 ('savefig.frameon : True')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.4/matplotlibrc.template
or from the matplotlib source distribution

Bad key nbagg.transparent in file c:\users\abhishek buragohaibn\anaconda3\envs\computervison\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 426 ('nbagg.tran

In [2]:
def fetch_data(data_dir):
    """
    laod all json formatted files into a dataframe
    """

    
    
    ## input testing
    if not os.path.isdir(data_dir):
        raise Exception("specified data dir does not exist")
    if not len(os.listdir(data_dir)) > 0:
        raise Exception("specified data dir does not contain any files")

    file_list = [os.path.join(data_dir,f) for f in os.listdir(data_dir) if re.search("\.json",f)]
    correct_columns = ['country', 'customer_id', 'day', 'invoice', 'month',
                       'price', 'stream_id', 'times_viewed', 'year']

    ## read data into a temp structure
    all_months = {}
    for file_name in file_list:
        df = pd.read_json(file_name)
        all_months[os.path.split(file_name)[-1]] = df

    ## ensure the data are formatted with correct columns
    for f,df in all_months.items():
        cols = set(df.columns.tolist())
        if 'StreamID' in cols:
             df.rename(columns={'StreamID':'stream_id'},inplace=True)
        if 'TimesViewed' in cols:
            df.rename(columns={'TimesViewed':'times_viewed'},inplace=True)
        if 'total_price' in cols:
            df.rename(columns={'total_price':'price'},inplace=True)

        cols = df.columns.tolist()
        if sorted(cols) != correct_columns:
            raise Exception("columns name could not be matched to correct cols")

    ## concat all of the data
    df = pd.concat(list(all_months.values()),sort=True)
    years,months,days = df['year'].values,df['month'].values,df['day'].values 
    dates = ["{}-{}-{}".format(years[i],str(months[i]).zfill(2),str(days[i]).zfill(2)) for i in range(df.shape[0])]
    df['invoice_date'] = np.array(dates,dtype='datetime64[D]')
    df['invoice'] = [re.sub("\D+","",i) for i in df['invoice'].values]
    
    ## sort by date and reset the index
    df.sort_values(by='invoice_date',inplace=True)
    df.reset_index(drop=True,inplace=True)
    
    return(df)

In [3]:

data_dir = "./cs-train/"
df = fetch_data(data_dir)

In [4]:
df.head(10)

country  customer_id  day invoice  month  price stream_id  \
0  United Kingdom      13085.0   28  489434     11   6.95     85048   
1  United Kingdom      13085.0   28  489434     11   6.75    79323W   
2  United Kingdom      13085.0   28  489434     11   2.10     22041   
3  United Kingdom      13085.0   28  489434     11   1.25     21232   
4  United Kingdom      13085.0   28  489434     11   1.65     22064   
5  United Kingdom      13085.0   28  489434     11   1.25     21871   
6  United Kingdom      13085.0   28  489434     11   5.95     21523   
7  United Kingdom      13085.0   28  489435     11   2.55     22350   
8  United Kingdom      13085.0   28  489435     11   3.75     22349   
9  United Kingdom      13085.0   28  489435     11   1.65     22195   

   times_viewed  year invoice_date  
0            12  2017   2017-11-28  
1            12  2017   2017-11-28  
2            21  2017   2017-11-28  
3             5  2017   2017-11-28  
4            17  2017   2017-11-28  
5            14  2017   2017-11-28  
6            10  2017   2017-11-28  
7            12  2017   2017-11-28  
8            12  2017   2017-11-28  
9            18  2017   2017-11-28

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 815011 entries, 0 to 815010
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   country       815011 non-null  object        
 1   customer_id   625249 non-null  float64       
 2   day           815011 non-null  int64         
 3   invoice       815011 non-null  object        
 4   month         815011 non-null  int64         
 5   price         815011 non-null  float64       
 6   stream_id     815011 non-null  object        
 7   times_viewed  815011 non-null  int64         
 8   year          815011 non-null  int64         
 9   invoice_date  815011 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(4), object(3)
memory usage: 62.2+ MB


In [6]:
df.isnull().sum()

country              0
customer_id     189762
day                  0
invoice              0
month                0
price                0
stream_id            0
times_viewed         0
year                 0
invoice_date         0
dtype: int64

In [7]:
df.describe().transpose()

count          mean          std       min       25%  \
customer_id   625249.0  15333.415068  1698.360788  12346.00  13956.00   
day           815011.0     15.064819     8.788845      1.00      7.00   
month         815011.0      6.592718     3.598781      1.00      3.00   
price         815011.0      4.802631   135.058707 -53594.36      1.25   
times_viewed  815011.0      5.231106     5.109982      0.00      1.00   
year          815011.0   2018.247654     0.545261   2017.00   2018.00   

                  50%       75%      max  
customer_id   15279.0  16813.00  18287.0  
day              15.0     23.00     31.0  
month             6.0     10.00     12.0  
price             2.1      4.21  38970.0  
times_viewed      3.0      8.00     24.0  
year           2018.0   2019.00   2019.0

In [8]:
df.describe(include =  np.object).transpose()

count unique             top    freq
country    815011     43  United Kingdom  751228
invoice    815011  42646          537434    1350
stream_id  815011   5007          85123A    5017